# Example 3: Extended usage and output diagnostics

In this notebook, we demonstrate extended usage of `pyvbmc`. We will:
- take a brief look at `pyvbmc`'s diagnostic output,
- show you how to continue optimization starting from the results of a previous run, and
- show you how to save the results of optimization to disk.

This notebook is Part 3 of a series of notebooks in which we present various example usages for VBMC with the `pyvbmc` package.

In [1]:
import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
import dill

## 1. Convergence Diagnostics

For demonstration purposes, we will run VBMC with a restricted budget of function evaluations, insufficient to achieve convergence. Then we will inspect the output diagnostics, and resume optimization.

We use a higher-dimensional analogue of the same toy target function in Example 1, a broad [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in $D = 4$.

In [2]:
D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)

Of course, your likelihood will in general depend on the observed data. This data can be fixed as a global variable, as we did above for `prior_mu` and `prior_var`. It can also be defined by a default keyword argument: to `pyvbmc` there is no difference so long as the function can be called with only a single argument (the parameters `theta`):

In [3]:
def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    return log_likelihood(theta, data) + log_prior(theta)

In [4]:
LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds

In a typical inference scenario, we recommend starting from a "good" point (i.e. one near the mode). We can run a  quick preliminary optimization (though a more extensive optimization would not harm).

In [5]:
np.random.seed(42)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(lambda t: -log_joint(t), x0).x

In [6]:
# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    user_options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )

Reshaping x0 to row vector.
Reshaping lower bounds to (1, 4).
Reshaping upper bounds to (1, 4).
Reshaping plausible lower bounds to (1, 4).
Reshaping plausible upper bounds to (1, 4).


In [7]:
vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         10          -9.40         4.43     28517.14        2        inf     start warm-up
     1         15          -5.33         1.04         3.02        2        inf     
     2         20          -4.54         2.02         3.79        2       72.6     
     3         25          -5.84         1.35        13.66        2        236     
     4         30          -3.74         1.56         5.13        2       97.7     
     5         35          -4.89         0.41         2.05        2       39.4     
     6         40          -4.95         0.19         0.17        2       3.68     
   inf         40          -4.68         0.19         0.26       50       3.68     finalize
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -4.679 +/-0.192.
Caution: Re

`pyvbmc` is warning us that convergence is doubtful. We can look at the output for more information and diagnostics.

In [8]:
success_flag

False

`False` means that `pyvbmc` has not converged to a stable solution within the given number of function evaluations.

In [9]:
info

{'function': '<function VBMC.__init__.<locals>.log_joint at 0x7fd217ebc280>',
 'problemtype': 'unconstrained',
 'iterations': 6,
 'funccount': 40,
 'bestiter': 6,
 'trainsetsize': 40,
 'components': 50,
 'rindex': 3.681110484591521,
 'convergencestatus': 'no',
 'overhead': nan,
 'rngstate': 'rng',
 'algorithm': 'Variational Bayesian Monte Carlo',
 'version': '0.0.1',
 'message': 'Inference terminated: reached maximum number of function evaluations options.max_fun_evals.',
 'elbo': -4.679438624214694,
 'elbo_sd': 0.19191071535555979}

In the `info` dictionary:
- the `convergencestatus` field says 'no' (probable lack of convergence);
- the reliability index `rindex` is 3.68, (should be less than 1).
Our diagnostics tell us that this run has not converged, suggesting to increase the budget. Note that convergence to a solution does not mean that it is a _good_ solution. You should always check the returned variational posteriors, and ideally should compare across multiple runs of `pyvbmc`.

## Continuing optimization from a previous result

We can continue running `pyvbmc` where we left off by calling the `optimize()` method again. But first we will change the function evaluation budget to a more realistic value.

In [10]:
# This verbose syntax is required to avoid unintentionally modifying
# options after initialization:
vbmc.options.__setitem__("max_fun_evals", 50 * (D + 2), force=True)
vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         50          -2.43         4.60         5.57        2        inf     
     1         55          -4.77         0.11         1.35        2        inf     
     2         60          -4.64         0.08         0.06        2       1.75     
     3         65          -4.61         0.02         0.07        3       1.39     
     4         70          -4.49         0.02         0.07        4        1.6     
     5         75          -4.44         0.01         0.02        5       0.43     
     6         80          -4.39         0.01         0.01        6      0.335     rotoscale, undo rotoscale
     7         85          -4.30         0.00         0.02        9      0.687     
     8         90          -4.27         0.00         0.00       12      0.164     
     9         95          -4.26         0.00   

(<pyvbmc.variational_posterior.variational_posterior.VariationalPosterior at 0x7fd217ca1760>,
 -4.138023411605985,
 0.0017356383397193903,
 True,
 {'function': '<function VBMC.__init__.<locals>.log_joint at 0x7fd217ebc280>',
  'problemtype': 'unconstrained',
  'iterations': 14,
  'funccount': 120,
  'bestiter': 14,
  'trainsetsize': 120,
  'components': 50,
  'rindex': 0.01949541499596393,
  'convergencestatus': 'probable',
  'overhead': nan,
  'rngstate': 'rng',
  'algorithm': 'Variational Bayesian Monte Carlo',
  'version': '0.0.1',
  'message': 'Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.',
  'elbo': -4.138023411605985,
  'elbo_sd': 0.0017356383397193903})

## 3. Saving results

We can also save the `VBMC` instance to disk and reload it later, in order to continue optimization, sample from the posterior, check the results, etc.

In [11]:
with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)

In [12]:
with open("vbmc_test_save.pkl", "rb") as f:
    vbmc_2 = dill.load(f)
vbmc_2.vp.sample(5)

(array([[ 0.14643708,  0.11663166, -0.4388296 ,  0.21406378],
        [ 0.1235108 ,  0.11301893, -0.33017338, -1.36320763],
        [-2.38341194, -0.01001572,  0.52415271,  1.58546946],
        [-1.97368552, -0.12112751,  0.00569015,  0.33803262],
        [-0.53941319, -0.2833481 , -0.02000113, -0.39568992]]),
 array([ 8,  8, 33,  2,  8]))

## 5. Conclusions

In this notebook, we have given a brief overview of `pyvbmc`'s output diagnostics, and shown how to save, load, and resume optimization. 

In the next notebook, we will illustrate running `pyvbmc` multiple times in order to validate the results.

## Example 3: full code

The following cell includes in a single place all the code used in Example 3, without the extra fluff.

In [13]:
assert False  # By default, don't run this cell
import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
import dill


D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)


def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    return log_likelihood(theta, data) + log_prior(theta)


LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds


np.random.seed(42)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(lambda t: -log_joint(t), x0).x


# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    user_options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )


vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()


success_flag


info


# This verbose syntax is required to avoid unintentionally modifying
# options after initialization:
vbmc.options.__setitem__("max_fun_evals", 50 * (D + 2), force=True)
vbmc.optimize()


with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)


with open("vbmc_test_save.pkl", "rb") as f:
    vbmc_2 = dill.load(f)
vbmc_2.vp.sample(5)

AssertionError: 

## Acknowledgments

Work on the `pyvbmc` package was funded by the [Finnish Center for Artificial Intelligence FCAI](https://fcai.fi/).